# Installing Relevant Packages for Reddit WebScraping

In [21]:
#pip install psaw

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
#pip install praw

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# Importing Libraries

In [2]:
from psaw import PushshiftAPI
import datetime as dt
from pprint import pprint
import pandas as pd
import numpy as np
from praw.models import MoreComments
import sys, os
import traceback
import praw
from IPython.display import clear_output

**Note: Please create a Reddit account, and enable developer mode to retrieve API token from https://www.reddit.com/prefs/apps**

In [3]:
user_agent = "Scrapper 1.0 by /u/Top_Transition3105"
reddit = praw.Reddit(
    client_id='XXXXXXXXXXXXXXXXXXXX',
    client_secret='XXXXXXXXXXXXXXXXXXXXXXXXXX',
    user_agent = user_agent,
    check_for_async=False
)

In [4]:
# PushshiftAPI allows us to obtain historical data from specified time range from Reddit
api = PushshiftAPI()

In [5]:
start_epoch=int(dt.datetime(2016, 1, 1).timestamp()) 
# year month day
end_epoch=int(dt.datetime(2022, 10, 2).timestamp())

title = set()

posts_dict = {"Subreddit": [], 'Post URL': [] ,"Date Created": [], "Title": [], "Self Text": [],
              "Score": [], "Upvote Ratio": [],
              "Total Comments": [], "Comments": []}

#For companies that are not so prevalent on Reddit, we expand our subreddit list to include more results
#subreddit_list = ['stocks', 'investing', 'StockMarket', 'wallstreetbets', 'politics', 'Stock_Picks', 'finance']

#For companies that are already prevalent on Reddit, we limit our scraping to the following subreddit
subreddit_list = ['stocks', 'StockMarket', 'wallstreetbets']

count = 0
f = open('nvidiaNLPUpdated.txt', 'a')

for subreddit_item in subreddit_list:

    current_date = 0
    date_count = 0
    time_before_6 = 0
    time_before_12 = 0
    time_before_18 = 0
    time_before_24 = 0
    
    for post in api.search_submissions(subreddit=subreddit_item, after = start_epoch, before=end_epoch):
        try:
            words_for_nvidia = ['nvidia', 'nvda', '$nvda']

            title = post.title
            try:
                text = post.selftext
            except Exception as e:
                text = ''

            for word in words_for_nvidia:
                if (word.lower() in title.lower()) or (word.lower() in text.lower()):
                    try:
                        date_time = pd.to_datetime(post.created, unit = 's', utc=True).tz_convert('Asia/Singapore')
                        date = date_time.date()
                        hour = date_time.hour
                        # print(date_time.hour)

                        if hour <=6:
                            time_before_6 += 1
                        elif hour > 6 and hour <=12:
                            time_before_12 += 1
                        elif hour > 12 and hour <=18:
                            time_before_18 += 1
                        elif hour > 18 and hour <24:
                            time_before_24

                        if current_date == date:
                            if date_count > 4:
                                continue
                        if date != current_date:
                            date_count = 0
                            current_date = date
                            time_before_6 = 0
                            time_before_12 = 0
                            time_before_18 = 0
                            time_before_24 = 0

                        if time_before_6 > 1 and hour <=6:
                            continue
                        elif time_before_12 > 1 and hour > 6 and hour <=12:
                            continue
                        elif time_before_18 > 1 and hour > 12 and hour <=18:
                            continue
                        elif time_before_24 > 1 and hour > 18 and hour <24:
                            continue

                        date_count += 1

                    except Exception as e:
                        print(traceback.format_exc())
                        pass  

                    posts_dict["Subreddit"].append(subreddit_item)

                    try:
                        # URL of each post
                        url = post.url
                        posts_dict["Post URL"].append(post.url)
                    except Exception as e:
                        url = None
                        posts_dict["Post URL"].append(None)
                        print(traceback.format_exc())
                        pass

                    try:
                        # Date of each post
                        date_time = pd.to_datetime(post.created, unit = 's', utc=True).tz_convert('Asia/Singapore')
                        print(date_time)
                        print(f"Subreddit: {subreddit_item}")
                        posts_dict["Date Created"].append(date_time)
                    except Exception as e:
                        date_time = None
                        posts_dict["Date Created"].append(None)
                        print(traceback.format_exc())
                        pass

                    try:
                        # Title of each post
                        # Checking if title contains meta
                        posts_dict["Title"].append(title)
                    except Exception as e:
                        posts_dict["Title"].append(None)
                        print(traceback.format_exc())
                        pass

                    try:
                        # Content of each post
                        content = post.selftext
                        posts_dict["Self Text"].append(content)
                    except Exception as e:
                        posts_dict["Self Text"].append(None)
                        print(traceback.format_exc())
                        pass

                    try:
                        # Score of each post
                        score = post.score
                        posts_dict["Score"].append(score)
                    except Exception as e:
                        posts_dict["Score"].append(None)
                        print(traceback.format_exc())
                        pass

                    try:
                        # Upvote Ratio of each post
                        upvote = post.upvote_ratio
                        posts_dict["Upvote Ratio"].append(upvote)
                    except Exception as e:
                        posts_dict["Upvote Ratio"].append(None)
                        print(traceback.format_exc())
                        pass

                    try:
                        # Num comments of each post
                        comments_no = post.num_comments
                        posts_dict["Total Comments"].append(comments_no)
                    except Exception as e:
                        posts_dict["Total Comments"].append(None)
                        print(traceback.format_exc())
                        pass

                    try:
                        #Extracting comments using reddit.submission("id") instead
                        # print(post.url)
                        raw_comments = []
                        submission = reddit.submission(post.id)
                        for top_level_comment in submission.comments:
                            if isinstance(top_level_comment, MoreComments):
                                continue
                            raw_comments.append(top_level_comment.body)
                        raw_comments = raw_comments[1:]
                        posts_dict["Comments"].append(raw_comments)
                    except Exception as e:
                        posts_dict["Comments"].append(None)
                        print(traceback.format_exc())
                        pass
                    #clear_output for memory purposes else jupyter notebook might crash from excessive printing
                    clear_output(wait=True)
                    count += 1
                    print(f"Number of data scraped: {count}")
                    now = dt.datetime.now()
                    current_time = now.strftime("%H:%M:%S")
                    print("Current Time =", current_time, "\n")
                    
                    conversion_dict = {'Post URL': url,"Date Created": [date_time], "Title": [title], "Self Text": [content],
                                       "Score": [score], "Upvote Ratio": [upvote], "Total Comments": [comments_no], "Comments": [raw_comments]}

                    conversion_dict = pd.DataFrame(conversion_dict)
                    output = conversion_dict.to_json(orient= 'index')
            
                    f.write(output)  # python will convert \n to os.linesep
                    f.write("\n")
                    
                    break
        except ConnectionError as e:
            print(str(e))
            print("********Portal New connection timed out***********")
            print("********Sleeping***********")
            time.sleep(30)
            print("********Resume***********")
            break

Number of data scraped: 2
Current Time = 10:31:03 

2022-09-30 14:49:57+08:00
Subreddit: stocks


KeyboardInterrupt: 

**Note: The above cell would work perfectly, please ignore the KeyboardInterrupt**

In [ ]:
reddit_result = pd.DataFrame(posts_dict)

In [ ]:
nvidia_NLP = reddit_result.copy()

In [ ]:
nvidia_NLP

In [ ]:
output = nvidia_NLP.to_json(orient= 'index')

In [ ]:
with open('nvidia_NLP.txt', 'w') as f:
    f.write(output)

In [ ]:
nvidia_NLP.info()